In [2]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

In [5]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from bespoke.inventory.analysis.shared import create_queries
%load_ext autoreload
%autoreload 2

In [3]:
COMPANY_NAME = 'BBF'
COMPANY_IDENTIFIER = 'BBF'
TRANSFER_PACKAGES_START_DATE = '2019-01-01'
SALES_TRANSACTIONS_START_DATE = '2019-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
    '10/31/2021',
]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0,
    'find_parent_child_relationships': False,
    'use_prices_to_fill_missing_incoming': True,
    'external_pricing_data_config': {
        'category_to_fixed_prices': {
            'Buds': {
                'grams': 10.0
            },
            'Infused (edible)': {
                'each': 4.0
            },
            'Infused (non-edible)': {
                'each': 3.0
            },
            'Vape Product': {
                'each': 3.0
            },
            'Concentrate (Bulk)': {
                'grams': 6.0
            },
            'Concentrate': {
                'grams': 7.0
            },
            'Raw Pre-Rolls': {
                'grams': 7.0,
                'pounds': 80.0
            },
            'Shake/Trim (by strain)': {
                'grams': 8.0
            }
        }
    }
}
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES.append(TODAY_DATE)
print('Today is {}'.format(TODAY_DATE))

Today is 11/08/2021


In [6]:
# Download packages, sales transactions, incoming / outgoing tranfers

company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)

company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)

In [7]:
company_incoming_transfer_packages_dataframe.columns

Index(['delivery_type', 'license_number', 'manifest_number', 'created_date',
       'received_datetime', 'shipper_facility_license_number',
       'shipper_facility_name', 'recipient_facility_license_number',
       'recipient_facility_name', 'shipment_type_name',
       'shipment_transaction_type', 'package_id', 'package_label', 'type',
       'source_package_labels', 'source_harvest_names',
       'shipment_package_state', 'is_testing_sample', 'is_trade_sample',
       'product_category_name', 'product_name', 'package_lab_results_status',
       'shipper_wholesale_price', 'shipped_quantity',
       'shipped_unit_of_measure', 'received_quantity',
       'received_unit_of_measure', 'receiver_wholesale_price',
       'item_unit_weight', 'item_unit_weight_unit_of_measure_name'],
      dtype='object')

In [10]:
company_incoming_transfer_packages_dataframe[[
    'product_category_name',
    'received_quantity',
    'received_unit_of_measure',
    'receiver_wholesale_price',
]].groupby(['product_category_name', 'received_unit_of_measure']).sum()

received_quantity  \
product_category_name  received_unit_of_measure                      
Buds                   Grams                            92147.8037   
                       Pounds                               3.1420   
Concentrate            Grams                              418.0000   
Concentrate (Each)     Each                               840.0000   
Infused (edible)       Each                              6801.0000   
Infused (non-edible)   Each                               146.0000   
Infused Beverage       Each                               864.0000   
Raw Pre-Rolls          Grams                             7063.2800   
Shake/Trim             Grams                             5562.0000   
Shake/Trim (by strain) Grams                            16968.0000   
Vape Product           Each                              6214.0000   

                                                 receiver_wholesale_price  
product_category_name  received_unit_of_measure                            
Buds                   Grams                                    295587.07  
                       Pounds                                     5500.00  
Concentrate            Grams                                     13760.00  
Concentrate (Each)     Each                                       7902.63  
Infused (edible)       Each                                      36594.69  
Infused (non-edible)   Each                                       5310.00  
Infused Beverage       Each                                       2160.00  
Raw Pre-Rolls          Grams                                     41386.50  
Shake/Trim             Grams                                         7.60  
Shake/Trim (by strain) Grams                                      6815.27  
Vape Product           Each                                      69825.08

In [12]:
company_incoming_transfer_packages_dataframe[[
    'product_category_name',
    'shipped_quantity',
    'shipped_unit_of_measure',
    'shipper_wholesale_price',
]].groupby(['product_category_name', 'shipped_unit_of_measure']).sum()

shipped_quantity  \
product_category_name  shipped_unit_of_measure                     
Buds                   Grams                          92099.3037   
                       Pounds                             5.8554   
Concentrate            Grams                            418.0000   
Concentrate (Each)     Each                             840.0000   
Infused (edible)       Each                            6801.0000   
Infused (non-edible)   Each                             146.0000   
Infused Beverage       Each                             864.0000   
Raw Pre-Rolls          Grams                           7063.2800   
Shake/Trim             Grams                           5585.0000   
Shake/Trim (by strain) Grams                          14698.0000   
                       Pounds                             5.0000   
Vape Product           Each                            6230.0000   

                                                shipper_wholesale_price  
product_category_name  shipped_unit_of_measure                           
Buds                   Grams                                  308387.04  
                       Pounds                                   9596.03  
Concentrate            Grams                                   13760.00  
Concentrate (Each)     Each                                     7902.63  
Infused (edible)       Each                                    36599.69  
Infused (non-edible)   Each                                     5310.00  
Infused Beverage       Each                                     2160.00  
Raw Pre-Rolls          Grams                                   41386.50  
Shake/Trim             Grams                                       7.60  
Shake/Trim (by strain) Grams                                    2815.27  
                       Pounds                                   4000.00  
Vape Product           Each                                    69855.08

In [14]:
company_incoming_transfer_packages_dataframe[
    (company_incoming_transfer_packages_dataframe['product_category_name'] == 'Shake/Trim') & (company_incoming_transfer_packages_dataframe['shipped_unit_of_measure'] == 'Grams')
]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,...,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name
548,INCOMING_FROM_VENDOR,MP281397,0000349905,2020-11-30,2020-11-30 22:41:45+00:00,RMD685-C,"Holistic Industries, Inc.",MP281397,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000042406: Chaos Kush #3 Trim,passed,1.9,1190.0,Grams,1184.0,Grams,1.9,None,None
549,INCOMING_FROM_VENDOR,MP281397,0000349905,2020-11-30,2020-11-30 22:41:45+00:00,RMD685-C,"Holistic Industries, Inc.",MP281397,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000042406: Chaos Kush #3 Trim,passed,1.9,1790.0,Grams,1790.0,Grams,1.9,None,None
550,INCOMING_FROM_VENDOR,MP281397,0000349905,2020-11-30,2020-11-30 22:41:45+00:00,RMD685-C,"Holistic Industries, Inc.",MP281397,Boston Bud Factory Inc.,Unaffiliated Transfer,...,Tropicanna Haze Trim,passed,1.9,620.0,Grams,618.0,Grams,1.9,None,None
551,INCOMING_FROM_VENDOR,MP281397,0000349905,2020-11-30,2020-11-30 22:41:45+00:00,RMD685-C,"Holistic Industries, Inc.",MP281397,Boston Bud Factory Inc.,Unaffiliated Transfer,...,Tropicanna Haze Trim,passed,1.9,1985.0,Grams,1970.0,Grams,1.9,None,None


In [20]:
result_dataframe = company_incoming_transfer_packages_dataframe[[
    'product_category_name',
    'received_quantity',
    'received_unit_of_measure',
    'receiver_wholesale_price',
]].groupby(['product_category_name', 'received_unit_of_measure']).sum().reset_index()

In [21]:
result_dataframe

,product_category_name,received_unit_of_measure,received_quantity,receiver_wholesale_price
0,Buds,Grams,92147.8037,295587.07
1,Buds,Pounds,3.1420,5500.00
2,Concentrate,Grams,418.0000,13760.00
3,Concentrate (Each),Each,840.0000,7902.63
4,Infused (edible),Each,6801.0000,36594.69
5,Infused (non-edible),Each,146.0000,5310.00
6,Infused Beverage,Each,864.0000,2160.00
7,Raw Pre-Rolls,Grams,7063.2800,41386.50
8,Shake/Trim,Grams,5562.0000,7.60
9,Shake/Trim (by strain),Grams,16968.0000,6815.27


In [22]:
result_records = result_dataframe.to_dict('records')

In [30]:
result_dict = {}

for result_record in result_records:
    product_category_name = result_record['product_category_name']
    unit_of_measure = result_record['received_unit_of_measure']
    unit_cost = result_record['receiver_wholesale_price'] / result_record['received_quantity']
    if product_category_name not in result_dict:
        result_dict[product_category_name] = {}
    if unit_of_measure not in result_dict[product_category_name]:
        result_dict[product_category_name][unit_of_measure] = {}
    result_dict[product_category_name][unit_of_measure] = unit_cost

# For each product category, if 'Pounds' unit does not exist but 'Grams' unit does...
# compute cost associated with 'Pounds' unit as 453.592x of cost associated with 'Grams' unit.
for product_category_name in result_dict.keys():
    if (
        'Grams' in result_dict[product_category_name] and
        result_dict[product_category_name]['Grams'] and
        'Pounds' not in result_dict[product_category_name]
    ):
        result_dict[product_category_name]['Pounds'] = result_dict[product_category_name]['Grams'] * 453.592

# If 'Concentrate (Bulk)' does not exist, assume costs associated with it are same as 'Concentrate'.
if 'Concentrate (Bulk)' not in result_dict and 'Concentrate' in result_dict:
    result_dict['Concentrate (Bulk)'] = result_dict['Concentrate']

result_dict

{'Buds': {'Grams': 3.207749486491558, 'Pounds': 1750.4774029280713},
 'Concentrate': {'Grams': 32.91866028708134, 'Pounds': 14931.640956937797},
 'Concentrate (Each)': {'Each': 9.407892857142858},
 'Infused (edible)': {'Each': 5.380780767534184},
 'Infused (non-edible)': {'Each': 36.36986301369863},
 'Infused Beverage': {'Each': 2.5},
 'Raw Pre-Rolls': {'Grams': 5.859388272870396, 'Pounds': 2657.7716454678284},
 'Shake/Trim': {'Grams': 0.0013664149586479683, 'Pounds': 0.6197948939230492},
 'Shake/Trim (by strain)': {'Grams': 0.40165429042904294,
  'Pounds': 182.18717290429043},
 'Vape Product': {'Each': 11.236736401673639},
 'Concentrate (Bulk)': {'Grams': 32.91866028708134,
  'Pounds': 14931.640956937797}}

In [25]:
company_incoming_transfer_packages_dataframe[
    (company_incoming_transfer_packages_dataframe['product_category_name'] == 'Infused (non-edible)') & (company_incoming_transfer_packages_dataframe['shipped_unit_of_measure'] == 'Each')
]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,...,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name
158,INCOMING_FROM_VENDOR,MR281525,0000688832,2021-08-11,2021-08-12 15:52:05+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00001103832: Origyn Sativa Moonrocks 1g,passed,735.0,21.0,Each,21.0,Each,735.0,None,None
235,INCOMING_FROM_VENDOR,MR281525,0000600025,2021-06-08,2021-06-09 20:22:14+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000977101: Sira No Flavor THC Tincture 900mg,passed,900.0,15.0,Each,15.0,Each,900.0,None,None
418,INCOMING_FROM_VENDOR,MR281525,0000458332,2021-02-22,2021-02-23 21:42:47+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000801102: Purient Plus Personal Lubricant ...,passed,525.0,15.0,Each,15.0,Each,525.0,None,None
516,INCOMING_FROM_VENDOR,MR281525,0000395835,2021-01-04,2021-01-05 21:46:55+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000172318: Purient Personal Lubricant 300mg,passed,300.0,10.0,Each,10.0,Each,300.0,None,None
670,INCOMING_FROM_VENDOR,MR281525,0000144623,2020-06-26,2020-06-29 19:10:55+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000172305: Nordic Goddess 1:1 Body Balm 500mg,passed,1925.0,55.0,Each,55.0,Each,1925.0,None,None
682,INCOMING_FROM_VENDOR,MR281525,0000144623,2020-06-26,2020-06-29 19:10:55+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000172318: Purient Personal Lubricant 300mg,passed,600.0,20.0,Each,20.0,Each,600.0,None,None
699,INCOMING_FROM_VENDOR,MR281525,0000115810,2020-06-04,2020-06-05 16:41:20+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000172318: Purient Personal Lubricant 300mg,passed,150.0,5.0,Each,5.0,Each,150.0,None,None
706,INCOMING_FROM_VENDOR,MR281525,0000115810,2020-06-04,2020-06-05 16:41:20+00:00,MP281303,"SIRA NATURALS, INC.",MR281525,Boston Bud Factory Inc.,Unaffiliated Transfer,...,M00000172305: Nordic Goddess 1:1 Body Balm 500mg,passed,175.0,5.0,Each,5.0,Each,175.0,None,None


In [29]:
company_incoming_transfer_packages_dataframe[
    (company_incoming_transfer_packages_dataframe['product_category_name'] == 'Concentrate (Bulk)')
]

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,...,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name
